# Setup environment

In [1]:
import os
import requests
import sqlite3

## Define parameters

In [2]:
ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))

API_KEY = "4INghUtThsIBWPTIcvfKyf0kNS6MtSXcC4R6mpNB"
BASE_URL = "https://api.nal.usda.gov/fdc/v1/"

# Fetch nutritional information

## Fetch food class names

In [3]:
FOOD256_DIR = os.path.join(os.path.abspath(os.sep), "Datasets", "food256")

class_names = []

with open(os.path.join(FOOD256_DIR, "category.txt")) as file:
    file.readline()
    for line in file.readlines():
        class_names.append(line.split('\t')[1].strip())
                           
class_names

['rice',
 'eels on rice',
 'pilaf',
 "chicken-'n'-egg on rice",
 'pork cutlet on rice',
 'beef curry',
 'sushi',
 'chicken rice',
 'fried rice',
 'tempura bowl',
 'bibimbap',
 'toast',
 'croissant',
 'roll bread',
 'raisin bread',
 'chip butty',
 'hamburger',
 'pizza',
 'sandwiches',
 'udon noodle',
 'tempura udon',
 'soba noodle',
 'ramen noodle',
 'beef noodle',
 'tensin noodle',
 'fried noodle',
 'spaghetti',
 'Japanese-style pancake',
 'takoyaki',
 'gratin',
 'sauteed vegetables',
 'croquette',
 'grilled eggplant',
 'sauteed spinach',
 'vegetable tempura',
 'miso soup',
 'potage',
 'sausage',
 'oden',
 'omelet',
 'ganmodoki',
 'jiaozi',
 'stew',
 'teriyaki grilled fish',
 'fried fish',
 'grilled salmon',
 'salmon meuniere',
 'sashimi',
 'grilled pacific saury',
 'sukiyaki',
 'sweet and sour pork',
 'lightly roasted fish',
 'steamed egg hotchpotch',
 'tempura',
 'fried chicken',
 'sirloin cutlet',
 'nanbanzuke',
 'boiled fish',
 'seasoned beef with potatoes',
 'hambarg steak',
 'ste

## Perform nutritional lookup for each food (using FoodData Central API)

In [4]:
nutritional_information = {}

for food in class_names:
    # seach FoodData Central with class name as search term
    url = BASE_URL+"search"
    params = {
        'api_key': API_KEY,
        'generalSearchInput': food
    }
    data = requests.get(url=url, params=params).json()
    
    # take nutritional information from first result
    if len(data['foods']) > 0:
        foodId = data['foods'][0]['fdcId']

        # query FoodDataCentral for report
        url = BASE_URL+str(foodId)
        params = {
            'api_key': API_KEY
        }
        data = requests.get(url=url, params=params).json()
        nutritional_information[food] = data['foodNutrients']
        
nutritional_information

KeyboardInterrupt: 

## Fetch densities of each food

In [ ]:
portion_information = {}

with open(os.path.join(ROOT_DIR, "nutrition", "portions.txt")) as file:
    file.readline()
    for line in file.readlines():
        portion_information[line.split('\t')[3].strip()] = {
            'density': float(line.split('\t')[1]),
            'depth': int(line.split('\t')[2])
        }
        
portion_information

# Write nutritional information to .db file

In [29]:
DATABASE_PATH = os.path.join(ROOT_DIR, "nutrition", "food.db")

connection = sqlite3.connect(DATABASE_PATH)
c = connection.cursor()

# define fields
c.execute('''CREATE TABLE food
             (id integer PRIMARY KEY NOT NULL,
             name text NOT NULL,
             energy integer NOT NULL DEFAULT 0,
             protein real NOT NULL DEFAULT 0,
             fats real NOT NULL DEFAULT 0,
             carbohydrates real NOT NULL DEFAULT 0,
             sugars real NOT NULL DEFAULT 0,
             density real NOT NULL DEFAULT 1,
             depth real NOT NULL DEFAULT 5
             )''')

# populate database
i = 1
for food in class_names:
    
    energy = 0
    protein = 0
    fats = 0
    carbs = 0
    sugars = 0
    density = portion_information[food]['density']
    depth = portion_information[food]['depth']
    
    for nutrient in nutritional_information.get(food, []):
        if nutrient['nutrient']['id'] == 1008:
            energy = nutrient['amount']
        elif nutrient['nutrient']['id'] == 1004:
            fats = nutrient['amount']
        elif nutrient['nutrient']['id'] == 1003:
            protein = nutrient['amount']
        elif nutrient['nutrient']['id'] == 1005:
            carbs = nutrient['amount']
        elif nutrient['nutrient']['id'] == 2000:
            sugars = nutrient['amount']
            
    c.execute("INSERT INTO food VALUES (?,?,?,?,?,?,?,?,?)", (i, food, energy, protein, fats, carbs, sugars, density, depth))
            
    i += 1

connection.commit()
connection.close()

## Test SQLite database

In [8]:
connection = sqlite3.connect(DATABASE_PATH)
c = connection.cursor()

t = ('french fries',)
c.execute('SELECT * FROM food WHERE name = ?', t)
print(c.fetchone())

connection.close()

(98, 'french fries', 155, 2.38, 4.76, 25.0, 1.19, 0.8, 2.0)
